In [1]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import os
import torch

In [2]:
kfold = 10
train_folds_df = pd.read_csv("deneme.csv", sep="|", encoding="utf-8")
train_folds_df.head()

,tweet,target
0,yahu internetim de sorun var turkcell tedas ye...,olumsuz
1,lifebox da eski mailim kayıtlı oldugu icin gir...,olumsuz
2,turkcell tv hizmetinizi kontrol edebilmemiz ic...,nötr
3,sevgili turkcell twitterı ucretsiz kullanıyors...,olumsuz
4,turkcell beni magdur ediyor kaan bey kaanterzi...,olumsuz


In [3]:
if 'kfold' not in train_folds_df.columns:
    kf = KFold(n_splits=kfold, shuffle=True, random_state=42)
    train_folds_df['kfold'] = -1
    for fold, (train_index, val_index) in enumerate(kf.split(train_folds_df)):
        train_folds_df.loc[val_index, 'kfold'] = fold

In [4]:
train_folds_df.head()

,tweet,target,kfold
0,yahu internetim de sorun var turkcell tedas ye...,olumsuz,2
1,lifebox da eski mailim kayıtlı oldugu icin gir...,olumsuz,7
2,turkcell tv hizmetinizi kontrol edebilmemiz ic...,nötr,5
3,sevgili turkcell twitterı ucretsiz kullanıyors...,olumsuz,1
4,turkcell beni magdur ediyor kaan bey kaanterzi...,olumsuz,6


In [5]:
le = LabelEncoder()
le.fit(train_folds_df['target'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping = {k: int(v) for k, v in le_name_mapping.items()}
print(le_name_mapping)
train_folds_df['labels'] = train_folds_df['target'].apply(lambda x: le_name_mapping[x])

{'nötr': 0, 'olumlu': 1, 'olumsuz': 2}


In [6]:
train_folds_df.head(10)

,tweet,target,kfold,labels
0,yahu internetim de sorun var turkcell tedas ye...,olumsuz,2,2
1,lifebox da eski mailim kayıtlı oldugu icin gir...,olumsuz,7,2
2,turkcell tv hizmetinizi kontrol edebilmemiz ic...,nötr,5,0
3,sevgili turkcell twitterı ucretsiz kullanıyors...,olumsuz,1,2
4,turkcell beni magdur ediyor kaan bey kaanterzi...,olumsuz,6,2
5,turkcell reklamı gibisin oyle bos oyle anlamsı...,olumsuz,3,2
6,turkcell star fizy istemedigim halde uye yapıl...,olumsuz,1,2
7,turkcell kullanma yın hayatımın en buyuk hatas...,olumsuz,4,2
8,turkcell gne kuredecem senin ulan bir gunde gb...,olumsuz,3,2
9,turkcell boyut atladı artık hic cekmiyorintern...,olumsuz,9,2


In [7]:
for fold in range(kfold):
    # Split data into train and validation sets
    train_df = train_folds_df[train_folds_df['kfold'] != fold]
    validation_df = train_folds_df[train_folds_df['kfold'] == fold]

    # Prepare the data for simpletransformers
    train_df = pd.DataFrame({
        'text': train_df['tweet'],  # Replace 'your_text_column' with the actual column name for text
        'labels': train_df['labels']
    })
    validation_df = pd.DataFrame({
        'text': validation_df['tweet'],  # Replace 'your_text_column' with the actual column name for text
        'labels': validation_df['labels']
    })

    # Initialize the ClassificationModel
    model = ClassificationModel('bert',
                                'dbmdz/bert-base-turkish-uncased',
                                num_labels=len(le_name_mapping.keys()),
                                use_cuda=True,  # Use CUDA
                                args={'reprocess_input_data': True,
                                      'overwrite_output_dir': True,
                                      'num_train_epochs': 15,
                                      'train_batch_size': 64,
                                      'fp16': False,
                                      'save_model_every_epoch': True,
                                      'save_eval_checkpoints': False,
                                      'output_dir': f'bert_model_fold_merge_{fold}',
                                      'save_steps': 0 })

/home/ensar/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

In [8]:
model.train_model(train_df)

  0%|          | 0/18705 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/293 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/293 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/293 [00:00<?, ?it/s]

...

Running Epoch 11 of 15:   0%|          | 0/293 [00:00<?, ?it/s]

Running Epoch 12 of 15:   0%|          | 0/293 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/293 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/293 [00:00<?, ?it/s]

(4395, 0.11266618514894662)

In [9]:
result, model_outputs, wrong_predictions = model.eval_model(validation_df)

  0%|          | 0/2078 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/260 [00:00<?, ?it/s]

In [10]:
predictions = model_outputs.argmax(axis=1)
actuals = validation_df['labels'].values

# Print classification report
print(metrics.classification_report(actuals, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.766     0.744     0.755       528
           1      0.734     0.767     0.750       309
           2      0.919     0.919     0.919      1241

    accuracy                          0.852      2078
   macro avg      0.806     0.810     0.808      2078
weighted avg      0.852     0.852     0.852      2078



In [11]:
 f1score = metrics.f1_score(actuals, predictions, average='macro')

In [12]:
filename = 'folds_merge_results.txt'
if os.path.exists(filename):
    append_write = 'a'  # Append if file exists
else:
    append_write = 'w'  # Create a new file if not
with open(filename, append_write) as line:
    line.write(f"Fold : {fold}  , f1-score : {f1score}" + '\n')

In [13]:
import json
wrong_predictions_data = []
for wp, pred in zip(wrong_predictions, predictions):
    wrong_predictions_data.append({
            "guid": wp.guid,
            "text_a": wp.text_a,
            "text_b": wp.text_b,
            "true_label": wp.label,
            "predicted_label": int(pred)
        })
    
with open(f'wrong_predictions_{fold}.json', 'w') as file:
    json.dump(wrong_predictions_data, file, indent=4)

In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(actuals, predictions)
print(cm)

[[ 393   59   76]
 [  47  237   25]
 [  73   27 1141]]
